# **Nhóm 5 - L14**
**Thành viên:**
1. Đoàn Ngọc Mai Thy - 050608200705
2. Đỗ Ngọc Quỳnh Vy - 050608200774
3. Nguyễn Ngọc Thanh Vy - 050608200783

# **CÂU 1:**

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
import itertools
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
# create the session
conf = SparkConf()
# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
def parse_line(line):
    split = line.split("\t")
    userID = int(split[0])
    if len(split) == 1:
        list_friends = []
    else:
        list_friends = list(map(lambda a: int(a), split[1].split(',')))
    return userID, list_friends

In [ ]:
def friendship_to_connection(f_o):
    userID = f_o[0]
    friends = f_o[1]
    connections = []
    for friendID in friends:
        password = (userID, friendID)
        if userID > friendID:
            password= (friendID, userID)
        connections.append((password, 0))  # they are friends, value=0
    for friendpair in itertools.combinations(friends, 2):
        friend_0 = friendpair[0]
        friend_1 = friendpair[1]
        password = (friend_0, friend_1)
        if friend_0 > friend_1:
            password = (friend_1, friend_0)
        connections.append((password, 1))  # they have mutual friends, value=1
    return connections

In [ ]:
def mutual_friend_count_to_recommendation(f):
    pair = f[0]
    friend0 = pair[0]
    friend1 = pair[1]
    noMutFriends = f[1]
    recommendation0 = (friend0, (friend1, noMutFriends))
    recommendation1 = (friend1, (friend0, noMutFriends))
    return [recommendation0, recommendation1]

In [ ]:
def recommendation_to_sorted_truncated(recommendations):
    recommendations = sorted(recommendations, key=lambda x: (-x[1], x[0]))
    result = map(lambda x: x[0], recommendations[:10])
    return list(result)

In [ ]:
# Đọc từ tệp văn bản
data = sc.textFile('soc-LiveJournal1Adj.txt')

In [ ]:
# Chuyển đổi mỗi dòng thành cặp (user_id, [friend_id_0, friend_id_1, ...])
friendship = data.map(parse_line).filter(lambda friend: friend[1] != '')#.filter(lambda friend: friend[0] < 1000) # Lấy 1000 mẫu để kiểm tra
print(friendship.take(1))

[(0, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94])]


In [ ]:
# Chuyển đổi sở hữu bạn thành các cặp ((user_id, friend_id), VALUE).
# VALUE = 0 => Các cặp đã là bạn.
# VALUE = 1 => Các cặp có bạn chung.
friend_edges = friendship.flatMap(friendship_to_connection)
friend_edges.cache()
print(friend_edges.take(1))

[((0, 1), 0)]


In [ ]:
# Lọc các cặp đã là bạn
mutualFriend = friend_edges.groupByKey().filter(lambda edge: 0 not in edge[1]).flatMap(lambda a: [(a[0],item) for item in a[1]]) 
print(mutualFriend.take(11))

[((34392, 34450), 1), ((34392, 34450), 1), ((34392, 34450), 1), ((34392, 34450), 1), ((34392, 34450), 1), ((8, 19068), 1), ((50, 4990), 1), ((3931, 8703), 1), ((32631, 43897), 1), ((32631, 43897), 1), ((29481, 31997), 1)]


In [ ]:
# Đếm số lượng bạn chung bằng cách cộng giá trị
mutualFriend_counts = mutualFriend.reduceByKey(lambda a, b: a + b)
print(mutualFriend_counts.take(1))

[((20003, 24475), 1)]


In [ ]:
# Tạo các đối tượng đề xuất, nhóm chúng theo key, sau đó sắp xếp và
recommendations = mutualFriend_counts.flatMap(mutual_friend_count_to_recommendation).groupByKey()
print(recommendations.take(22))

[(39238, <pyspark.resultiterable.ResultIterable object at 0x7f42b4095df0>), (17558, <pyspark.resultiterable.ResultIterable object at 0x7f42b402e430>), (4674, <pyspark.resultiterable.ResultIterable object at 0x7f42b402e070>), (9248, <pyspark.resultiterable.ResultIterable object at 0x7f42b402e610>), (30090, <pyspark.resultiterable.ResultIterable object at 0x7f42b402efa0>), (10412, <pyspark.resultiterable.ResultIterable object at 0x7f42b402e640>), (34664, <pyspark.resultiterable.ResultIterable object at 0x7f42b402e730>), (40402, <pyspark.resultiterable.ResultIterable object at 0x7f42b402ea90>), (24134, <pyspark.resultiterable.ResultIterable object at 0x7f42b402eb20>), (44976, <pyspark.resultiterable.ResultIterable object at 0x7f42b402e880>), (4456, <pyspark.resultiterable.ResultIterable object at 0x7f42b4036370>), (27870, <pyspark.resultiterable.ResultIterable object at 0x7f42b4036b20>), (31838, <pyspark.resultiterable.ResultIterable object at 0x7f42b4036400>), (23296, <pyspark.resultiter

In [ ]:
# Cắt bớt các đề xuất thành 10 đề xuất được đề xuất nhiều nhất.
top10_recommendations = recommendations.map(lambda m: (m[0], recommendation_to_sorted_truncated(list(m[1])))).sortByKey() 

In [ ]:
# Bao gồm các đề xuất trong bài viết cho người dùng có ID sau: 924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993.
results = top10_recommendations.filter(lambda recommendations: recommendations[0] in [924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993])

In [ ]:
results.collect()

[(924, [439, 2409, 6995, 11860, 15416, 43748, 45881]),
 (8941, [8943, 8944, 8940]),
 (8942, [8939, 8940, 8943, 8944]),
 (9019, [9022, 317, 9023]),
 (9020, [9021, 9016, 9017, 9022, 317, 9023]),
 (9021, [9020, 9016, 9017, 9022, 317, 9023]),
 (9022, [9019, 9020, 9021, 317, 9016, 9017, 9023]),
 (9990, [13134, 13478, 13877, 34299, 34485, 34642, 37941]),
 (9992, [9987, 9989, 35667, 9991]),
 (9993, [9991, 13134, 13478, 13877, 34299, 34485, 34642, 37941])]